# Compare training a single model for multiple horizons, versus horizon-dedicated models
Normally in OpenSTEF, a single model is trained to forecast load at continuously increasing lead time.

However, let's analyse how the accuracy of this setup compares to training models for specific lead times.

Conclusion; on the very short term (15 minutes ahead), training a single model for that specific horizon is more accurate. For the longer time horizon, the model trained on both horizons actually outperforms the model only trained on that specific horizon.

In [ ]:
import pandas as pd
import cufflinks
cufflinks.go_offline()

from pathlib import Path
from datetime import datetime
import os
import yaml

from openstef.pipeline.train_create_forecast_backtest import train_model_and_forecast_back_test
from openstef.metrics.figure import plot_feature_importance
from openstef.data_classes.model_specifications import ModelSpecificationDataClass
from openstef.data_classes.prediction_job import PredictionJobDataClass

# Set working dir to location of this file
os.chdir('.')

In [ ]:
# Define name
nb_title = 'Compare_single_vs_multihorizon_model'

# Load inputs
filename = Path("../.data/Middenmeer-150kV.csv")

measurements = pd.read_csv(filename, delimiter=";", decimal=",")
measurements["Datetime"] = pd.to_datetime(measurements["Datum"] + " " + measurements["Tijd"])
measurements = measurements.set_index('Datetime').tz_localize('CET', ambiguous='NaT', nonexistent='NaT').tz_convert("UTC")
# Only keep relevant columns
measurements = measurements.iloc[:,2:-1]
# Sum the load
measurements['Total'] = measurements.sum(axis=1)
# By default, only a backtest is made for the total
target_column = 'Total'

measurements.iplot(layout=dict(template='plotly_white'))

In [ ]:
# Load predictors
predictors = pd.read_csv('../.data/predictors.csv', index_col=0, parse_dates=True)
predictors.head()

In [ ]:
# Define properties of training/prediction. We call this a 'prediction_job' 
pj=PredictionJobDataClass(
    id=1,
    name='TestPrediction',
    model='xgb',
    quantiles=[0.10,0.30,0.50,0.70,0.90],
    horizon_minutes=24*60,
    resolution_minutes=15,
        
    forecast_type="demand", # Note, this should become optional
    lat = 1, #should become optional
    lon = 1, #should become optional
                  )

training_horizons=[0.25, 47.0]

# Make backtest using a single model for all lead times
# Define backtest specs
backtest_specs = dict(n_folds=3, training_horizons=training_horizons)
modelspecs = ModelSpecificationDataClass(id=pj['id'])

# Specify input data, use last column of the load dataframe
input_data = pd.DataFrame(dict(load=measurements.loc[:,target_column])).merge(predictors, left_index=True, right_index=True)
# Also resample to fix overlapping indices
input_data = input_data.resample('15T').mean()


# Perform the backtest
forecast_single_model, model_single_model, train_data, validation_data, test_data = train_model_and_forecast_back_test(
    pj,
    modelspecs = modelspecs,
    input_data = input_data,
    **backtest_specs,
 )

# Store the model, so it can be compared to the other models
models=dict(multihorizonmodel=model_single_model)

In [ ]:
# Repeat backtest, but now with seperate models for each horizon
forecast_dedicated_model = pd.DataFrame()
models_dedicated_model = dict()
for horizon in training_horizons:
    forecast, model, train_data, validation_data, test_data = train_model_and_forecast_back_test(
        pj,
        modelspecs = modelspecs,
        input_data = input_data,
        **dict(n_folds=backtest_specs['n_folds'], training_horizons=[horizon]),
    )
    forecast_dedicated_model = forecast_dedicated_model.append(forecast)
    models.update({f'dedicated_model_{horizon}': model})

# Evaluate results

In [ ]:
# Combine. df should have the P50 forecast for single/multimodel and for short/long horizon
df = pd.DataFrame(dict(forecast_multihorizonmodel_short=forecast_single_model[forecast_single_model.horizon==0.25]['forecast'].values,
                       forecast_multihorizonmodel_long =forecast_single_model[forecast_single_model.horizon==47.0]['forecast'].values,
                       forecast_dedicatedmodels_short = forecast_dedicated_model[forecast_dedicated_model.horizon==0.25]['forecast'].values,
                       forecast_dedicatedmodels_long = forecast_dedicated_model[forecast_dedicated_model.horizon==47.0]['forecast'].values,
                       realised = forecast_dedicated_model[forecast_dedicated_model.horizon==47.0]['realised'].values,
                       ),
                  index = forecast_dedicated_model[forecast_dedicated_model.horizon==47.0].index.values)

In [ ]:
df.iplot()

In [ ]:
err_df = df.apply(lambda x: x-x.realised, axis=1)
err_df.iloc[:,:-1].abs().mean()[[0,2,1,3]].iplot(kind='bar', yTitle='MAE')

In [ ]:
## Plot feature importances of models - size = gain, color = weight.

for name, model in models.items():
    print(f'Name: {name}')
    feature_importance_fig = plot_feature_importance(model[0].feature_importance_dataframe)
    feature_importance_fig.show()

More information on feature importance; the size indicates gain, the color indicates weight. References: [ref1](https://datascience.stackexchange.com/questions/12318/how-to-interpret-the-output-of-xgboost-importance) [ref2](https://towardsdatascience.com/be-careful-when-interpreting-your-features-importance-in-xgboost-6e16132588e7)

The Gain is the most relevant attribute to interpret the relative importance of each feature.

‘Gain’ is the improvement in accuracy brought by a feature to the branches it is on. The idea is that before adding a new split on a feature X to the branch there was some wrongly classified elements, after adding the split on this feature, there are two new branches, and each of these branch is more accurate (one branch saying if your observation is on this branch then it should be classified as 1, and the other branch saying the exact opposite).

‘weight’: the number of times a feature is used to split the data across all trees.



# Store results
Store timeseries as csv, metadata as yaml, model as ... and write an overview to pdf.

In [ ]:
run_name = f'{datetime.utcnow():%Y%m%d_%H%M%S}_{nb_title}'

In [ ]:
def write_artifacts(run_name, forecast, model, prediction_job, backtest_specs):
    """Write timeseries to csv and generate PDF of result"""
    
    # Create output dir
    outdir = Path(f'output/{run_name}')
    if not os.path.exists(outdir):
        os.mkdir(outdir)
     
    # Write forecast_df (includes realised)
    forecast.to_csv(outdir / 'forecast.csv', compression='gzip')
    
    # Write model
    model.save_model(outdir / "model.json")
    
    # Write meta data - prediction job and backtest parameters
    # relevant prediction_job attributes
    rel_attrs = ['id','name','model','quantiles']
    rel_pj_dict={key:prediction_job[key] for key in rel_attrs}
    with open(outdir / "configs.yaml", "w") as file:
        documents = yaml.dump({**rel_pj_dict, **backtest_specs}, file)

write_artifacts(run_name, forecast, model[0], pj, backtest_specs)

In [ ]:
nb_fname = '02.Compare_single_vs_multihorizon_model'
command=f"jupyter nbconvert {nb_fname}.ipynb --to html --output results/{nb_title}.html"
os.system(command)

In [ ]:
command